In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import os
from scipy import stats, integrate
import seaborn as sns
sns.set(color_codes=True)
sns.set_style('whitegrid')

**MAH SHIT STARTS HERE**
![](https://media.giphy.com/media/9e1VPfBaNvvEY/giphy.gif)
The idea is to test whether the erratum is critical or not (ie false positive st. significant diff or *just* dismissing actual difs).

Besides training on pandas ofc

Pretty basic stuff, probably not written in a very 'pandorable' way.

In [ ]:
# Holm-Bonferroni / sequential Bonferroni method
# more info: https://en.wikipedia.org/wiki/Holm%E2%80%93Bonferroni_method#Example
def Holm(alpha, nUnderAlpha):
    '''nUnderAlpha must be a list containing p-values'''
    outHolm=[]
    nUnderAlpha.sort() # sort ascending
    holmvec = []
    for i in range(0, len(nUnderAlpha), 1):
        holmvec.append(alpha/(len(nUnderAlpha)-i))
        if nUnderAlpha[i]<=holmvec[i]:
            outHolm.append(nUnderAlpha[i])
        else:
            break # stops after the first occurrence of: p-value > adj alpha
    return outHolm # returns p-values that meet the criteria

In [ ]:
Holm(0.05, [0.05, 0.001, 0.012, 0.042]) # check that the function is actually working as expected

In [ ]:
# weird method
def homemadeAdjust(alpha, nUnderAlpha):
    '''Currently used method. nUnderAlpha must be a list containing p-values'''
    outHPA=[]
    nUnderAlpha.sort(reverse=True)
    HPAvec = []
    for i in range(0, len(nUnderAlpha), 1):
        HPAvec.append(alpha/(2**i)) # difficulty to meet the criteria rises exponentially
        if nUnderAlpha[i]<=HPAvec[i]:
            outHPA.append(nUnderAlpha[i])
            # However, there is no STOP in our current method. We just exclude the ones that do not meet the criteria
    return outHPA

In [ ]:
# example, which already makes no sense (like some log10 transf, but that's another story :p)
homemadeAdjust(0.05, [0.05, 0.001, 0.012, 0.042])

It seems obvious that the exclusion profiles are way different, regardless Holm-Bonferroni stopping when a given p-value does not meet exclusion criteria
[i am plotting them as an exercise]

In [ ]:
# generate some arrays to plot them (up to 20 comparisons)
hpamethod = []
holmethod = []
for i in range(0, 100, 1):
    holmethod.append(0.05/(1+ (i/5))) # added more points so it looks smoother
    hpamethod.append(0.05/(2**(i/5))) # SAME                  
    
#plot adjusted alpha 
plt.plot(np.arange(0, 10, 0.1),holmethod, color='blue') # range(0,20,1),
plt.plot(np.arange(0, 10, 0.1),hpamethod, color='red')
plt.xlabel('# prev comparison(s)')
plt.ylabel('adjusted alpha err thr')
plt.legend(['Holm-Bonferroni adj', 'Homemade adj'])
plt.show()

I am not very good at mental math and all that stuff, so I think it'd be better for all of us if we do a brief simulation to extract some conclusions (ie which one is more conservative)

In [ ]:
# try to keep it easy. More or less like prev funct
def Holm(v):
    vout = []
    v.sort()
    for i in range(0,len(v),1):
        if v[i]<=(0.05/(len(v)-i)):
            vout.append(v[i])
        else:
            break
    return vout

def Home(v):
    vout = []
    v.sort(reverse=True)
    for i in range(0,len(v),1):
        if v[i]<=(0.05/2**i):
            vout.append(v[i])
        else:
            pass
    return vout

def removed(v, r):
    '''returns v after removing r items'''
    if len(r)>0:
        for item in r:
            v.remove(item)
    return v

def randomprobvec(iternumber):
    randomvec = []
    for i in range(0, iternumber, 1):
        randomvec.append(list(np.random.uniform(0.00001, 0.05, [np.random.randint(2, 10)])))
    return randomvec # returns a list with length = random (from 2 to 10), whose items are values from 0.00001 to 0.05

#randomprobvec(2) #test, uncomment if you wish :)      

In [ ]:
def Holm_1(v): # returns p-values rejected by holm
    kek = v.copy()
    vout = Holm(kek)
    if len(vout)<1:
        return kek
    elif len(vout)==1:
        kek.remove(vout[0])
        return kek
    else:
        for item in vout:
            kek.remove(item)
        return kek
def Home_1(v): # returns p-values rejected by home
    kek = v.copy()
    vout = Home(kek)
    if len(vout)<1:
        return kek
    elif len(vout)==1:
        kek.remove(vout[0])
        return kek
    else:
        for item in vout:
            kek.remove(item)
        return kek

In [ ]:
prob = pd.Series(randomprobvec(10000)) # generate 10k prob vecs using the previous function
holm = pd.Series(prob.apply(Holm)) # prob vecs after prunning with holm
home = pd.Series(prob.apply(Home)) # same but home

In [ ]:
# organize data in a single dataframe
df=pd.DataFrame([prob, holm, home])
df=df.T
df.columns=['prob', 'holm','home']
df.head()

In [ ]:
print(df.iloc[0]['prob']) # just to display the whole lists 
print(df.iloc[0]['holm'])
print(df.iloc[0]['home'])

In [ ]:
df['holm-1']=pd.Series(prob.apply(Holm_1)) # these new columns contain values rejected by each method
df['home-1']=pd.Series(prob.apply(Home_1))
df.head()

In [ ]:
# visualize a row of the dataframe & check functions are working.
rowtocheck = 1
print(df.iloc[rowtocheck]['prob']) 
print(df.iloc[rowtocheck]['holm'])
print(df.iloc[rowtocheck]['home'])
print(df.iloc[rowtocheck]['holm-1'])
print(df.iloc[rowtocheck]['home-1'])

In [ ]:
for column in ['prob', 'holm', 'home']: # return new series containing the length of the lists
    df['len'+column] = df[column].apply(lambda x: len(x))
df.head()

In [ ]:
for column in ['lenprob', 'lenholm', 'lenhome']:
    print(column+' mean: '+str(df[column].mean())+' | sd: '+str(df[column].std())+' | SEM: '+str(df[column].sem())+' | TOT: '+str(df[column].sum())) # heh 10k rows -_-

In [ ]:
# show resulting vector length distribution after each method. Ignore x-axis label
plt.figure('distr')
sns.distplot(df['lenholm'])
sns.distplot(df['lenhome'])
plt.legend(['lenholm', 'lenhome'])
plt.show()

In [ ]:
# try to get a vector containing holm-1 and home-1 discarded pvalues (holm-1 positive, home-1 negative, if both = 0). So later we can plot them
def exclusions(holm, home):
    a = holm.copy()
    b = home.copy()
    outvec = []
    if len(a)>0 and len(b)>0:
        for itema in a:
            if itema in b:
                outvec.append(0)
                b.remove(itema)
                #a.remove(itema)
            else:
                outvec.append(itema)
                a.remove(itema)
        if len(b)>0:
            for itemb in b:
                outvec.append((itemb*(-1)))
            return outvec
        else:
            return outvec
    elif len(a)>0 and len(b)==0:
        return a
    elif len(a)==0 and len(b)>0:
        return [x * (-1) for x in b]
    else:
        return []
            

In [ ]:
df['difs'] = df.apply(lambda x: exclusions(x['holm-1'],x['home-1']), axis=1)
df.head()

In [ ]:
def remove0(l):
    l1 = l.copy()
    return list(filter((0).__ne__, l))
df['difs0']= df['difs'].apply(lambda x: remove0(x))
df.head()

In [ ]:
rowtocheck = 1
print(df.iloc[rowtocheck]['prob'])
print(df.iloc[rowtocheck]['holm'])
print(df.iloc[rowtocheck]['home'])
print(df.iloc[rowtocheck]['holm-1'])
print(df.iloc[rowtocheck]['home-1'])
print(df.iloc[rowtocheck]['difs'])
print(df.iloc[rowtocheck]['difs0'])

In [ ]:
def scatteraggregator(vecdfcol, originalLen): # a funct to append datapoints. Not very readable
    '''need to fix dif and dif0 first bcause they are not giving what we expect// done
    so x should be len(prob) and Y1 = dif0>0 ; and Y2 abs(dif0<0)'''
    holm = []
    home = []
    for item in vecdfcol:
        if item>0:
            holm.append(item)
        elif item<0:
            home.append(abs(item)) # absolute value to plot it in the same quadrant
    return originalLen, holm, home # returns a tuple of (int, list, list)

In [ ]:
scatteraggregator(df['difs0'][1], df['lenprob'][1]) # is it working?

In [ ]:
def scatplt(idx): # function designed to fill some lists/arrays so we feed matplotlib
    testing1x = []
    testing2x = []
    testing1y = []
    testing2y = []
    for item in scatteraggregator(df['difs0'][idx], df['lenprob'][idx])[1]: # holm y values
        testing1x.append(scatteraggregator(df['difs0'][idx], df['lenprob'][idx])[0]+np.random.uniform(-0.2, 0.2, size=None))# adding some horizontal jitter
        testing1y.append(item) # holm y values
    for item in scatteraggregator(df['difs0'][idx], df['lenprob'][idx])[2]:
        testing2x.append(scatteraggregator(df['difs0'][idx], df['lenprob'][idx])[0]+np.random.uniform(-0.2, 0.2, size=None))
        testing2y.append(item)
    return testing1x, testing2x, testing1y, testing2y

In [ ]:
## try to plot points in the same prev plot
# generate some arrays to plot them (up to 20 comparisons)
%pylab inline
pylab.rcParams['figure.figsize'] = (16, 9) # matplotlib graph size stuff
hpamethod = []
holmethod = []
for i in range(0, 100, 1):
    holmethod.append(0.05/(1+ (i/5))) # added more points so it looks smoother
    hpamethod.append(0.05/(2**(i/5))) # SAME                  

#plot it
plt.plot(np.arange(0, 10, 0.1),holmethod, color='blue') # lines again
plt.plot(np.arange(0, 10, 0.1),hpamethod, color='red')
bluecounter = 0
redcounter = 0 # so whe know how many datapoints we have at the end
for i in range(500):#number df rows to plot , the more the slower :)
    plt.scatter(scatplt(i)[0], scatplt(i)[2], s=20, facecolors='none', edgecolors="blue") # rejected pvalues ONLY by holm
    plt.scatter(scatplt(i)[1], scatplt(i)[3], s=20, facecolors='none', edgecolors="red") # rejected pvalues ONLY by home
    bluecounter += len(scatplt(i)[2])
    redcounter += len(scatplt(i)[3])
plt.xlabel('# prev comparison(s)')
plt.ylabel('adjusted p-value')
plt.legend(['Holm-Bonferroni adj', 'Homemade adj'])
plt.show()
print('Only rejected by Holm method #Blue = '+str(bluecounter)+'\nOnly rejected by Home method#Red = '+str(redcounter))

To conclude:
Although conservativeness increases exponentially in home-method, the fact that it does not stop causes it to be less conservative than holm's and to eat rubbish around 0.05

kill me please.

**ERRATUM**

since lines are based on # prev comparisons and points in length they do not match in the x-axis. Easy to notice because home method cannot discard all pvalues under 0.05.

e.g. imagine [0.05, 0.04]
* Holm:  0.04 > (0.05/2); dump both
* Home: 0.05 is <= (0.05/1), welcome; 0.04 > (0.05/2), get rekt!    ¯\\\_(ツ)\_/¯

[re-plotting them with x-1 in scatterplot so it match]

In [ ]:
def scatplt(idx): # function designed to fill some lists/arrays so we feed matplotlib
    testing1x = []
    testing2x = []
    testing1y = []
    testing2y = []
    for item in scatteraggregator(df['difs0'][idx], df['lenprob'][idx])[1]: # holm y values
        testing1x.append(scatteraggregator(df['difs0'][idx], df['lenprob'][idx])[0]+np.random.uniform(-0.2, 0.2, size=None)-1)# adding some horizontal jitter
        testing1y.append(item) # holm y values
    for item in scatteraggregator(df['difs0'][idx], df['lenprob'][idx])[2]:
        testing2x.append(scatteraggregator(df['difs0'][idx], df['lenprob'][idx])[0]+np.random.uniform(-0.2, 0.2, size=None)-1)
        testing2y.append(item)
    return testing1x, testing2x, testing1y, testing2y

%pylab inline
pylab.rcParams['figure.figsize'] = (16, 9) # matplotlib graph size stuff
hpamethod = []
holmethod = []
for i in range(0, 100, 1):
    holmethod.append(0.05/(1+ (i/5))) # added more points so it looks smoother
    hpamethod.append(0.05/(2**(i/5))) # SAME                  

#plot it
plt.plot(np.arange(0, 10, 0.1),holmethod, color='blue') # lines again
plt.plot(np.arange(0, 10, 0.1),hpamethod, color='red')
bluecounter = 0
redcounter = 0 # so whe know how many datapoints we have at the end
for i in range(500):#number df rows to plot , the more the slower :)
    plt.scatter(scatplt(i)[0], scatplt(i)[2], s=20, facecolors='none', edgecolors="blue") # rejected pvalues ONLY by holm
    plt.scatter(scatplt(i)[1], scatplt(i)[3], s=20, facecolors='none', edgecolors="red") # rejected pvalues ONLY by home
    bluecounter += len(scatplt(i)[2])
    redcounter += len(scatplt(i)[3])
plt.xlabel('# prev comparison(s)')
plt.ylabel('adjusted p-value')
plt.legend(['Holm-Bonferroni adj', 'Homemade adj'])
plt.show()
print('Only rejected by Holm method #Blue = '+str(bluecounter)+'\nOnly rejected by Home method#Red = '+str(redcounter))

**other stuff**

In [ ]:
# animal experimental grups being from 8 to 15?
group_size = [8, 15]
xax = np.arange(group_size[0], group_size[1]+1, step=1)
testlen = group_size[1]
ttest_pvec = []
a = np.random.normal(850, 200, testlen)
b = np.random.normal(850, 200, testlen)
#print(a)
#print(b)
for i in range(group_size[1]-group_size[0]+1):
    _, p = stats.ttest_ind(a[:i+group_size[1]-group_size[0]],b[:i+group_size[1]-group_size[0]])
    ttest_pvec += [float(p)]
plt.plot(xax, ttest_pvec)
plt.axhline(y=0.05, color='r', linestyle='--')
plt.xlabel('group size')
plt.ylabel('p value')
plt.show()

In [ ]:
# now avoid plots and gather some simulations reusing the same code
simulation_size = 10000
pval_arr = np.empty(0)
group_size = [2, 15]
for i in range(simulation_size):
    a = np.random.normal(850, 200, group_size[1])
    b = np.random.normal(850, 200, group_size[1])
    ttest_pvec = []
    for j in range(group_size[1]-group_size[0]+1):
        _, p = stats.ttest_ind(a[:j+group_size[0]],b[:j+group_size[0]])
        ttest_pvec += [float(p)]
    
    pval_arr = np.append(pval_arr, ttest_pvec) 
#pval_arr


In [ ]:
#pval_arr.reshape(2,8)
total_cols = group_size[1]-group_size[0]+1
arr_to_tweak = pval_arr.reshape(simulation_size, total_cols)
perc_sig = ['x']*total_cols
for i in range(total_cols):
    amount_sig = len(np.where(arr_to_tweak[:,i:i+1]<0.05)[0])
    perc_sig[i] = round(100*amount_sig/simulation_size,2)
perc_sig


In [ ]:
# same test but not sequential picking, just fully random
# now avoid plots and gather some simulations reusing the same code
simulation_size = 10000
pval_arr = np.empty(0)
ef_size_arr = np.empty(0)
group_size = [2, 30]
for i in range(simulation_size):
    ttest_pvec = []
    ef_size_vec = []
    for j in range(group_size[1]-group_size[0]+1):
        a = np.random.normal(850, 200, j+group_size[0])
        b = np.random.normal(850, 200, j+group_size[0])
        _, p = stats.ttest_ind(a,b)
        ef_size_vec += [np.around(np.abs(np.mean(a)-np.mean(b)),1)]
        ttest_pvec += [float(p)]
    ef_size_arr = np.append(ef_size_arr, ef_size_vec)
    pval_arr = np.append(pval_arr, ttest_pvec) 
total_cols = group_size[1]-group_size[0]+1
ef_tweak = ef_size_arr.reshape(simulation_size, total_cols)
arr_to_tweak = pval_arr.reshape(simulation_size, total_cols)
perc_sig = ['x']*total_cols
mean_eff = ['m']*total_cols
for i in range(total_cols):
    spur = np.where(arr_to_tweak[:,i:i+1]<0.05)
    amount_sig = len(spur[0])
    perc_sig[i] = round(100*amount_sig/simulation_size,2)
    mean_eff[i] = np.mean(ef_tweak[spur])
#print(perc_sig)
#print(mean_eff)
plt.figure(figsize=(8, 10))
plt.subplot(211)
plt.plot(np.arange(group_size[0], group_size[1]+1, step=1), perc_sig)
plt.axhline(y=np.mean(perc_sig), color='r', linestyle='--')
plt.title('alpha errors')
plt.xlabel('group size')
plt.ylabel('perc statistically significant')
plt.subplot(212)
plt.plot(np.arange(group_size[0], group_size[1]+1, step=1), mean_eff)
plt.title('effect size')
plt.xlabel('group size')
plt.show()


afterall this t-test is rly good :)
Let's see what happens when there are actual difs

In [ ]:
simulation_size = 1000
group_size = [2, 30]
lines = 6
with plt.style.context(('seaborn-colorblind')):
    plt.figure(figsize=(14, 10))
    for h in range(lines):
        pval_arr = np.empty(0)
        ef_size_arr = np.empty(0)
        for i in range(simulation_size):
            ttest_pvec = []
            ef_size_vec = []
            for j in range(group_size[1]-group_size[0]+1):
                a = np.random.normal(850, 200, j+group_size[0])
                b = np.random.normal(850-85*h, 200-20*h, j+group_size[0])
                _, p = stats.ttest_ind(a,b)
                ef_size_vec += [np.around(np.abs(np.mean(a)-np.mean(b)),1)]
                ttest_pvec += [float(p)]
            ef_size_arr = np.append(ef_size_arr, ef_size_vec)
            pval_arr = np.append(pval_arr, ttest_pvec) 
        total_cols = group_size[1]-group_size[0]+1
        ef_tweak = ef_size_arr.reshape(simulation_size, total_cols)
        arr_to_tweak = pval_arr.reshape(simulation_size, total_cols)
        perc_sig = ['x']*total_cols
        mean_eff = ['m']*total_cols
        for i in range(total_cols):
            spur = np.where(arr_to_tweak[:,i:i+1]<0.05)
            amount_sig = len(spur[0])
            perc_sig[i] = round(100*amount_sig/simulation_size,2)
            mean_eff[i] = np.mean(ef_tweak[spur])
        plt.plot(np.arange(group_size[0], group_size[1]+1, step=1), perc_sig)
    plt.axhline(y=50, color='r', linestyle='--')
    plt.axhline(y=5, color='b', linestyle=':')
    plt.title('prob statistically significant vs groupsize')
    plt.xlabel('group size')
    plt.ylabel('% statistically significant')
    legendlist=[]
    for i in range(lines):
        legendlist += ['dif-'+str(i*10)+'%'] 
    plt.legend(legendlist)
plt.show()

In [ ]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/wBig6rYT88I?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')
# iframe? fkin zangief players, fk banishing flat aka greenhand
# for the memes and the lulz

*Qué onda banda, acabo de darle una correteada a un animal, que alguien me diga que animal es este, miren esta raro, me lo encontre, yo pensé que era un pato, veanle las patas válgame, HEEEEY QUIETO PUTO, NO ME ESTES PICOTEANDO, mi jefa tiene un zoológico en su casa, se lo voy a llevar a ella, pero vean es un animal raro, AAAAAAAAH ME PICÓ HIJO DE SU PINCHE MADRE, UUOOOH ME ESTA PICANDO BANDA, AAAAAAH CULERO AAAAAAAAH PUTO SUELTAME A LA VERGA AAAAAAH *